<a href="https://colab.research.google.com/github/bacetiner/BrailsPlusPlus/blob/master/examples/image_processor/construction_type_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install BRAILS++**
BRAILS++ is not yet available on PyPi. The following line installs the latest version from the GitHub repository using `pip`.

In [ ]:
pip install git+https://github.com/NHERI-SimCenter/BrailsPlusPlus

## **Import ImageSet and Importer Methods of BRAILS++**
BRAILS++ offers various Artificial Intelligence-powered modules for predicting building and infrastructure attributes from images, such as the RoofShapeClassifier demonstrated in this example.

The `Importer` class provides the main methods to load these modules. The `ImageSet` class and BRAILS++ scrapers dedicated to this purpose can be utilized to get the images needed to predict attributes. This example will illustrate both approaches.

In [ ]:
from brails.types.image_set import ImageSet
from brails.utils.utils import Importer

## **Load Street-Level Imagery: (Option 1) Download Imagery by Specifying the Region Name**
The `ConsTypeClassifier` class is created to identify the roof types of buildings using satellite or aerial imagery. In this option, BRAILS++ retrieves the required imagery by following the steps below.
1. Call the `RegionBoundary` class with the necessary details to specify the region of interest. In this example, bird's eye imagery of buildings in Pinellas County, FL are downloaded. Therefore, the required information includes the `type` of information provided and the `data` itself, which are the `locationName` and `Alameda Island, CA`.
2. Download the footprint inventory for the `RegionBoundary` saved in `region_boundary_object` by downloading the OpenStreetMap footprint data through the `OSM_FootprintScraper`. The only information required to initialize `OSM_FootprintScraper` is the desired output unit for `length`, which, in this case, is `ft` . The output of `OSM_FootprintScraper` is an `AssetInventory` saved in `osm_fp_inventory`.
3. Get a randomly selected `100` building subset of the footprint inventory obtained using `OSM_FootprintScraper`. This subset is obtained using the `get_random_sample` method of an `AssetInventory` object and is stored in `alameda_subset`. For this example, the random seed is set to `40` for reproducability of the results.
4. Get the street-level imagery for the selected subset


In [ ]:
# Select a region and create its RegionBoundary:
region_data = {"type": "locationName",
               "data": "Alameda Island, CA"}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

osm_fp_class = importer.get_class("OSM_FootprintScraper")
osm_fp_data = {"length": "ft"}
footprint_scraper = osm_fp_class(osm_fp_data)
alameda_fp = footprint_scraper.get_footprints(region_boundary_object)

# Subsample from the extracted assets to keep the image downloading step quick.
# Here, we are randomly sampling 20 buildings using a random seed value of 40:
alameda_fp_subset = alameda_fp.get_random_sample(100, 40)

# Get aerial imagery for the selected subset using GoogleSatellite:
google_street_class = importer.get_class("GoogleStreet")
google_street = google_street_class()
images_street = google_street.get_images(
    alameda_fp_subset, "tmp/street/")

## **Predict Building Occupancies Using ConsTypeClassifier Module**
`ConsTypeClassifier` includes a pre-trained EfficientNet_V2_M model that has been trained on a custom dataset of 100,000 images. This model is capable of classifying roofs seen in aerial imagery into five categories: `Concrete`, `Wood`, `Steel`, `Masonry`, and `Manufactured`.

In [ ]:
importer = Importer()
constype_classifier_class = importer.get_class('ConsTypeClassifier')
constype_classifier = constype_classifier_class()
predictions = constype_classifier.predict(images_street)

## **Load Street Imagery**
The `RoofShapeClassifier` class is designed to identify roof types of buildings based on aerial imagery. In this example, we will load images from the `examples/image_processor/images/satellite_easy` directory of BRAILS++ to showcase how to use the `ConsTypeClassifier`.

To load these images, you first need to initialize an `ImageSet` object, which will be assigned to aerial_images. This object can then load the target images by scanning the specified directory using the set_directory method. The second argument in set_directory is necessary for loading the images; otherwise, the default behavior of `ImageSet` is to simply set the directory to the string value provided in the first argument (in this case, `images/satellite`).

## **Retrain Default OccupancyClassifier Module**

In [ ]:
constype_classifier.retrain(images_street)